In [24]:
#papermill_description=imports

import json
import os
import logging
import sys
import geopandas as gpd
from io import StringIO
import openmeteo_requests
import requests_cache
import pandas as pd
from datetime import datetime
from gis_utils.dataframe import get_bbox_from_geodf
from retry_requests import retry

# Configure logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
#papermill_description=openmeteo_api_setup

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [18]:
def calculate_days_between(date_str1: str, date_str2: str, format = "%Y-%m-%d") -> int:
    """
    Calculate the number of days between two dates given as strings.

    Parameters:
    - date_str1 (str): The first date string.
    - date_str2 (str): The second date string.

    Returns:
    - int: The difference in days between the two dates.
    
    Example:
    ```python
    days_difference = calculate_days_between('2024-05-01', '2024-04-25')
    print(f"The difference in days is: {days_difference}")
        
    """
    # convert the date strings into datetime objects
    date_format = format  # allow for custom date formats because merica
    datetime1 = datetime.strptime(date_str1, date_format)
    datetime2 = datetime.strptime(date_str2, date_format)
    
    # calculate the difference in days
    delta = datetime1 - datetime2
    return abs(delta.days)  # use abs to ensure a non-negative results

In [26]:
#papermill_description=parameters

model = "weather"
notebook_key = "localjupyter"
geojson = {
    'body': {
				"type": "FeatureCollection",
				"name": "Wooreen B (Lot 2 PS626962)",
				"crs": {
					"type": "name",
					"properties": {
						"name": "urn:ogc:def:crs:OGC:1.3:CRS84"
					}
				},
				"features": [
					{
						"type": "Feature",
						"geometry": {
							"type": "Polygon",
							"coordinates": [
								[
									[
										146.01870666235175,
										-38.41014685435973
									],
									[
										146.01156329529925,
										-38.40605842401869
									],
									[
										146.01250874093756,
										-38.4050294202201
									],
									[
										146.0134366783259,
										-38.40564682425665
									],
									[
										146.01345418657837,
										-38.40586634442194
									],
									[
										146.0143120909538,
										-38.4062916628427
									],
									[
										146.0150299293088,
										-38.40555078397516
									],
									[
										146.01439963221662,
										-38.405207781927956
									],
									[
										146.01401445066034,
										-38.40531754276017
									],
									[
										146.01285890599144,
										-38.404617814599
									],
									[
										146.01338421183698,
										-38.40255975863582
									],
									[
										146.01587038370303,
										-38.40293021434827
									],
									[
										146.01634310652224,
										-38.40284789102072
									],
									[
										146.01714848614,
										-38.40146210093232
									],
									[
										146.01742861818093,
										-38.40140721760968
									],
									[
										146.01825150605123,
										-38.40224418375359
									],
									[
										146.01975721577412,
										-38.4026009204754
									],
									[
										146.0207726944226,
										-38.403053699163905
									],
									[
										146.02213833812493,
										-38.4034653136932
									],
									[
										146.02329445989193,
										-38.403740192078175
									],
									[
										146.024327446793,
										-38.403795073629475
									],
									[
										146.0249402356352,
										-38.40384995513907
									],
									[
										146.01870666235175,
										-38.41014685435973
									]
								]
							]
						},
						"properties": {}
					}
				]
		}
}
propertyName = "test"
output_type = "weather"
datetime_from="2024-04-01"
datetime_end="2024-05-23"


In [28]:
#papermill_description=process_variables

santized_datetime_from = datetime_from.replace(":", "").replace("-", "").replace(" ", "")
santized_datetime_end = datetime_end.replace(":", "").replace("-", "").replace(" ", "")
weather_output_daily_filename = f"/tmp/{model}/{output_type}_{propertyName}_{santized_datetime_from}_{santized_datetime_end}weather.csv"
weather_output_hourly_filename = f"/tmp/{model}/{output_type}_{propertyName}_{santized_datetime_from}_{santized_datetime_end}weather.csv"

In [29]:
#papermill_description=processing_file_io

req = geojson
geojson_data = req['body']  # Directly accessing the 'body' since it's already a dictionary in this mock setup

# Convert the GeoJSON string to a GeoDataFrame
gdf = gpd.read_file(StringIO(json.dumps(geojson_data)))

INFO:botocore.credentials:Found credentials in environment variables.


In [30]:
#papermill_description=processing_bounding_box

# Get bounding box from GeoJSON
bbox = get_bbox_from_geodf(geojson_data)

# get the central latitude and longitude of the bounding box
gpd_lon = (bbox[0] + bbox[2]) / 2
gpd_lat = (bbox[1] + bbox[3]) / 2
centroid = [gpd_lon, gpd_lat]

In [31]:
def process_weather_data(weather_data, variable_order):
    variables = {name: weather_data.Variables(index).ValuesAsNumpy() for index, name in enumerate(variable_order)}
    time_range = pd.date_range(
        start=pd.to_datetime(weather_data.Time(), unit="s", utc=True),
        end=pd.to_datetime(weather_data.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=weather_data.Interval()),
        inclusive="left"
    )
    data = {"date": time_range}
    data.update(variables)
    return pd.DataFrame(data)

In [33]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below

# Example usage
days_difference = calculate_days_between(datetime_from, datetime_end)
print(f"The difference in days is: {days_difference}")

# for wint speed, direction etc. the 10m and 40m are metres above surface level, not a cap on the maximum values recordable.
url = "https://api.open-meteo.com/v1/bom"

params = {
    "latitude": 52.52,
    "longitude": 13.41,
    "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "showers", "snowfall", 
                "weather_code", "cloud_cover", "et0_fao_evapotranspiration", "wind_speed_10m", "wind_speed_40m"],
    "daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", 
              "apparent_temperature_min", "sunrise", "sunset", "daylight_duration", "sunshine_duration", 
              "uv_index_max", "uv_index_clear_sky_max", "precipitation_sum", "rain_sum", "showers_sum", 
              "snowfall_sum", "precipitation_hours", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"],
    "past_days": 92,
    "past_hours": 24,
    "forecast_hours": 24
}

responses = openmeteo.weather_api(url, params=params)

response = responses[0]

hourly_dataframe = process_weather_data(response.Hourly(), params['hourly'])
daily_dataframe = process_weather_data(response.Daily(), params['daily'])

print(hourly_dataframe)
print(daily_dataframe)

The difference in days is: 52
                        date  temperature_2m  relative_humidity_2m  \
0  2024-05-22 06:00:00+00:00       19.965000                  66.0   
1  2024-05-22 07:00:00+00:00       20.815001                  66.0   
2  2024-05-22 08:00:00+00:00       21.765001                  63.0   
3  2024-05-22 09:00:00+00:00       22.615000                  62.0   
4  2024-05-22 10:00:00+00:00       23.415001                  61.0   
5  2024-05-22 11:00:00+00:00       23.565001                  67.0   
6  2024-05-22 12:00:00+00:00       22.865000                  64.0   
7  2024-05-22 13:00:00+00:00       23.165001                  64.0   
8  2024-05-22 14:00:00+00:00       22.315001                  72.0   
9  2024-05-22 15:00:00+00:00       21.915001                  72.0   
10 2024-05-22 16:00:00+00:00       21.815001                  72.0   
11 2024-05-22 17:00:00+00:00       21.665001                  73.0   
12 2024-05-22 18:00:00+00:00       21.265001                